#### Functions for text formatting

In [11]:
RESET = "\033[0m"
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
COLOR_RED = "\033[31m"
COLOR_GREEN = "\033[32m"
COLOR_CYAN = "\033[36m"

def textf(text, format):
    return f"{format}{text}{RESET}"

def bold(text):
    return textf(text, BOLD)

def underline(text):
    return textf(text, UNDERLINE)

# Anomaly Detection using K-Means and Spectral Clustering

## First: Importing the necessary libraries

In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pickle

## Second: Importing the dataset and preprocessing it

- We first load the data from the CSVs.
- We then proceed with preprocessing the data as follows:
    1. Split the labels from the data (last column) and place them in separate dataframes.
    2. Encode the categorical data using into one-hot vectors using the OneHotEncoder class.
    3. Scale the features using the StandardScaler class.

In [13]:
def load_data() -> tuple:

    # Loading the data as indicated in the assignment
    # kddcup.data.corrected is the training data, corrected is the testing data
    print('Loading the data...', end=' ')
    training_data = pd.read_csv( 'archive/kddcup.data_10_percent_corrected', header=None )
    testing_data = pd.read_csv( 'archive/corrected', header=None )
    print( textf('Done!', COLOR_GREEN) )

    # Separating the labels (last column) from the features in the training and testing data
    print('Separating the labels from the features...', end=' ')
    X_train = training_data.iloc[:, :-1]
    y_train = training_data.iloc[:, -1]

    X_test = testing_data.iloc[:, :-1]
    y_test = testing_data.iloc[:, -1]
    print( textf('Done!', COLOR_GREEN) )

    # Concatenating the training and testing data vertically to perform OneHotEncoding on all the categorical features
    print('Concatenating the training and testing data...', end=' ')
    X = pd.concat( [X_train, X_test], axis=0 )
    print( textf('Done!', COLOR_GREEN) )

    # Encoding the features into one-hot vectors using OneHotEncoder
    # ColumnTransformer is used to apply the OneHotEncoder to the second, third and fourth columns (categorical features)
    # The remainder is set to 'passthrough' to keep the other columns unchanged (already numerical)
    print('Encoding the features into one-hot vectors...', end=' ')
    ct = ColumnTransformer( [('one_hot_encoder', OneHotEncoder(), [1, 2, 3])], remainder='passthrough' )
    ct = ct.fit(X)
    X_train = pd.DataFrame( ct.transform(X_train) )
    X_test = pd.DataFrame( ct.transform(X_test) )
    print( textf('Done!', COLOR_GREEN) )

    # Feature Scaling since some features have a much higher range than others
    print('Feature Scaling...', end=' ')
    scaler = StandardScaler()
    X_train = pd.DataFrame( scaler.fit_transform( X_train ) )
    X_test = pd.DataFrame( scaler.transform( X_test ) )
    print( textf('Done!', COLOR_GREEN) )

    print('All done!')

    return X_train, y_train, X_test, y_test

# Randomly picking some samples from the training data to speed up the training process (for testing purposes only)
def sample_data(X_train, y_train, X_test, y_test, n_train:int=10000, n_test:int=1000) -> tuple:
    print('Sampling the data...', end=' ')
    X_train = X_train.sample(n_train, random_state= 42)
    y_train = y_train[X_train.index]

    X_test = X_test.sample(n_test, random_state= 42)
    y_test = y_test[X_test.index]
    print( textf('Done!', COLOR_GREEN) )
    return X_train, y_train, X_test, y_test

In [14]:
X_train, y_train, X_test, y_test = load_data()

Loading the data... Done!
Separating the labels from the features... Done!
Concatenating the training and testing data... Done!
Encoding the features into one-hot vectors... Done!
Feature Scaling... Done!
All done!


In [15]:
# OPTIONAL: Uncomment the following line to sample the data (for testing purposes only)

# X_train, y_train, X_test, y_test = sample_data(X_train, y_train, X_test, y_test)

In [16]:
try:
    print('Number of unique labels in the training data: ', len(y_train.unique()))
    print('-' * 50)
    print('Number of unique labels in the testing data : ', len(y_test.unique()))
except NameError:
    print( textf('NameError: y_train or y_test is not defined', COLOR_RED) )

Number of unique labels in the training data:  23
--------------------------------------------------
Number of unique labels in the testing data :  38


## Third: Applying K-Means and Normalized Cut

### Supplementary Functions

In [17]:
# This function saves the kmeans model state using pickle
def save_state_kmeans( k: int, centroids: pd.DataFrame, clusters: dict, cluster_indices: dict, cluster_labels: dict ):
    with open('checkpoints/centroids' + str(k) + '.pkl', 'wb') as file:
        pickle.dump(centroids, file)
    with open('checkpoints/clusters' + str(k) + '.pkl', 'wb') as file:
        pickle.dump(clusters, file)
    with open('checkpoints/cluster_indices' + str(k) + '.pkl', 'wb') as file:
        pickle.dump(cluster_indices, file)
    with open('checkpoints/cluster_labels' + str(k) + '.pkl', 'wb') as file:
        pickle.dump(cluster_labels, file)

# This function loads the kmeans model state using pickle
def load_state_kmeans( k: int ) -> tuple:
    with open('checkpoints/centroids' + str(k) + '.pkl', 'rb') as file:
        centroids = pd.DataFrame( pickle.load(file) )
    with open('checkpoints/clusters' + str(k) + '.pkl', 'rb') as file:
        clusters = dict( pickle.load(file) )
    with open('checkpoints/cluster_indices' + str(k) + '.pkl', 'rb') as file:
        cluster_indices = dict( pickle.load(file) )
    with open('checkpoints/cluster_labels' + str(k) + '.pkl', 'rb') as file:
        cluster_labels = dict( pickle.load(file) )
    return centroids, clusters, cluster_indices, cluster_labels

### K-Means Clustering Algorithm

#### Implementation

In [18]:
def kmeans_clustering( k, data, max_iterations:int=None, print_updates=False, initial_centroids=None, save_cp=True ):
    
    # Initially, selecting k random data points as centroids
    # We will use the current time as the seed to make sure that we get different centroids each time we run the algorithm
    
    np.random.seed( 42 )

    if initial_centroids is None:
        centroids = data.sample( k, random_state= 42 )
    else:
        centroids = initial_centroids.copy()
        
    old_centroids = None

    # If the user doesn't specify the maximum number of iterations, we will set it to infinity (Loop until convergence)
    if max_iterations is None:
        max_iterations = np.inf

    itr = 1
    while( itr <= max_iterations ):

        # If the centroids do not change, we will stop the algorithm
        if centroids.equals( old_centroids ):
            break

        # Storing the old centroids to check if they change in the next iteration
        old_centroids = centroids.copy()

        if print_updates is True: print( underline(bold(' Iteration #' + str(itr) + ' ')) )

        # Container initialization for cluster data
        clusters = {} # clusters[i] will store the data points in the (i+1)th cluster
        cluster_indices = {} # cluster_indices[i] will store the indices of the data points in the (i+1)th cluster
        cluster_labels = {} # cluster_labels[i] will store the label of the (i+1)th cluster by majority voting
        for i in range(k):
            clusters[i] = []
            cluster_indices[i] = []

        # Broadcasting the centroids and the data points to make the calculations easier
        # centroids is a (k x d) matrix, data is a (n x d) matrix
        # So, we will broadcast them both to a (n x k x d) matrix in order to be able to calculate the distances between them
        centroids_broadcasted = np.broadcast_to( centroids.to_numpy(), (data.shape[0], k, data.shape[1]) )
        data_broadcasted = np.broadcast_to( data.to_numpy()[:, np.newaxis, :], (data.shape[0], k, data.shape[1]) )

        # Calculating the distances between the data points and the centroids
        # We use axis=2 because we want to calculate the distance between each data point and each centroid along the feature axis
        distances = np.linalg.norm( data_broadcasted - centroids_broadcasted, axis=2 )

        # Finding the closest centroid for each data point
        # We use axis=1 because we want to find the closest centroid for each data point along the centroid axis
        closest_cluster_indices = np.argmin( distances, axis=1 )


        # Assigning the data points to the clusters
        for i in range(k):
            clusters[i] = data.iloc[ closest_cluster_indices == i ]
            cluster_indices[i] = data.index[ closest_cluster_indices == i ].tolist()

        # Updating the centroids.
        # We will use our calculate_mean function to calculate the mean of the data points in the cluster
        # because it handles both numerical and categorical data
        for i in range(k):

            # If the cluster is empty, we will not update the centroid
            if len(clusters[i]) == 0:
                continue
            else:
                centroids.iloc[i] = np.mean( clusters[i], axis=0 )

        
        # Calculating the cluster labels by majority voting (if the cluster is not empty)
        for i in range(k):
            if len(clusters[i]) == 0:
                cluster_labels[i] = None
            else:
                cluster_labels[i] = pd.Series( [y_train[index] for index in cluster_indices[i]] ).value_counts().index[0]

        # Printing the cluster sizes and labels in a pandas table
        if print_updates is True:
            print( pd.DataFrame( [ [len(clusters[i]), cluster_labels[i]] for i in range(k) ], columns=['Cluster Size', 'Cluster Label'] ) )

        if print_updates is True: print('-' * 50) # Just to print a line to separate the iterations

        if save_cp: save_state_kmeans( k, centroids, clusters, cluster_indices, cluster_labels )

        itr += 1

    return centroids, clusters, cluster_indices, cluster_labels

#### Execution

In [19]:
# This function will be used to calculate the purity of the clusters
# Purity is the percentage of data points in a cluster that belong to the same class
def calculate_purity( clusters, labels, print_report=False ):
    purities = []
    for i in range( len(clusters) ):
        cluster = clusters[i]

        # If the cluster is empty, we will skip it
        if len(cluster) == 0: continue

        # Converting the cluster to a dataframe so that we can use the value_counts() function
        cluster = pd.DataFrame(cluster)
        cluster['label'] = labels[cluster.index]

        # We will use the value_counts() function to count the number of data points in each class
        # and then we will divide it by the total number of data points in the cluster
        purities.append( cluster['label'].value_counts()[0] / len(cluster) )

    # Normalizing the purity by dividing it by the number of clusters
    average_purity = sum(purities) / len(clusters)

    if print_report is True:
        for i in range(len(purities)):
            print('Cluster ', i+1, ' purity: ', purities[i])
        print('-'*50)
        print('Average Purity: ', average_purity)
        print('-'*50)
    
    return average_purity, purities


# This function prints a report of the clusters produced by the k-means algorithm
def analyze_clusters( clusters, cluster_indices, cluster_labels, labels ):

    # Printing the number of data points in each cluster
    for i in range(len(cluster_indices)):
        print('Cluster ', i+1, ' contains ', len(cluster_indices[i]), ' data points of class ', cluster_labels[i])
    print( '-' * 50 )

    # Calculating the purity of the clusters and printing the report
    calculate_purity( clusters, labels, print_report=True )

    # Printing the count for each unique labels in each cluster horizontally
    for i in range(len(cluster_indices)):
        print( bold('[Cluster #' + str(i+1) + ']'), ' --> ', textf(cluster_labels[i], COLOR_CYAN) )
        print(labels[cluster_indices[i]].value_counts())
        print('-' * 50)

In [20]:
try:
    centroids7, clusters7, cluster_indices7, cluster_labels7 = kmeans_clustering( k=7, data=X_train, print_updates=True, save_cp=False )
except KeyboardInterrupt:
    print( textf('Process interrupted by user', COLOR_RED) )

 Iteration #1 
   Cluster Size Cluster Label
0        300767        smurf.
1             0          None
2             0          None
3         83600       normal.
4         92831      neptune.
5             0          None
6         16823      neptune.
--------------------------------------------------
 Iteration #2 
   Cluster Size Cluster Label
0         18092       normal.
1        280256        smurf.
2             0          None
3         84812       normal.
4         25048      neptune.
5             0          None
6         85813      neptune.
--------------------------------------------------
 Iteration #3 
   Cluster Size Cluster Label
0         21011       normal.
1         27685        smurf.
2        253586        smurf.
3         76526       normal.
4         28326      neptune.
5             0          None
6         86887      neptune.
--------------------------------------------------
 Iteration #4 
   Cluster Size Cluster Label
0         20987       normal.
1      

In [21]:
try:
    centroids15, clusters15, cluster_indices15, cluster_labels15 = kmeans_clustering( k=15, data=X_train, print_updates=True, save_cp=False )
except KeyboardInterrupt:
    print( textf('Process interrupted by user', COLOR_RED) )

 Iteration #1 
    Cluster Size Cluster Label
0         227577        smurf.
1              0          None
2              0          None
3          46922       normal.
4          33415      neptune.
5              0          None
6          13093      neptune.
7          30452       normal.
8              0          None
9          38813       normal.
10         34534        smurf.
11             0          None
12         40705      neptune.
13         22765      neptune.
14          5745       normal.
--------------------------------------------------
 Iteration #2 
    Cluster Size Cluster Label
0            158        smurf.
1         226449        smurf.
2              0          None
3          43462       normal.
4           7761      neptune.
5              0          None
6          40060      neptune.
7          31704       normal.
8              0          None
9          21287       normal.
10         54518        smurf.
11             0          None
12         39234    

In [34]:
centroids23, clusters23, cluster_indices23, cluster_labels23 = load_state_kmeans(23)

In [35]:
try:
    centroids23, clusters23, cluster_indices23, cluster_labels23 = kmeans_clustering( k=23, data=X_train, print_updates=True, initial_centroids=centroids23 )
except KeyboardInterrupt:
    print( textf('Process interrupted by user', COLOR_RED) )

 Iteration #1 
    Cluster Size Cluster Label
0            327          pod.
1            355       normal.
2           5330       normal.
3          14061       normal.
4           1659      neptune.
5            110       normal.
6          46626      neptune.
7          59377       normal.
8           9200        smurf.
9          19972       normal.
10         34627        smurf.
11          1544       normal.
12         36691      neptune.
13         21909      neptune.
14          1642      ipsweep.
15           202      neptune.
16           864      neptune.
17           163          pod.
18           745      neptune.
19          9907        smurf.
20           735      neptune.
21        226998        smurf.
22           977     teardrop.
--------------------------------------------------
 Iteration #2 
    Cluster Size Cluster Label
0            275          pod.
1            368       normal.
2           5327       normal.
3          14073       normal.
4           1655    

In [ ]:
try:
    centroids31, clusters31, cluster_indices31, cluster_labels31 = kmeans_clustering( k=31, data=X_train, print_updates=True )
except KeyboardInterrupt:
    print( textf('Process interrupted by user', COLOR_RED) )

 Iteration #1 
    Cluster Size Cluster Label
0          56939       normal.
1           2179        smurf.
2              0          None
3           3160        smurf.
4              0          None
5            601        smurf.
6          17667        smurf.
7            984        smurf.
8           3185        smurf.
9              0          None
10         10360        smurf.
11         45058        smurf.
12          4221        smurf.
13         19607       normal.
14             0          None
15         24317        smurf.
16             0          None
17          9248        smurf.
18          2222        smurf.
19          5537        smurf.
20             0          None
21            62        smurf.
22          3602        smurf.
23         10652        smurf.
24          9413        smurf.
25         55494        smurf.
26          6619       normal.
27             0          None
28          6579        smurf.
29             0          None
30         13323        

In [ ]:
try:
    centroids45, clusters45, cluster_indices45, cluster_labels45 = kmeans_clustering( k=45, data=X_train, print_updates=True )
except KeyboardInterrupt:
    print( textf('Process interrupted by user', COLOR_RED) )

 Iteration #1 
    Cluster Size Cluster Label
0          56939       normal.
1           1294        smurf.
2              0          None
3           3160        smurf.
4              0          None
5            600        smurf.
6          12925        smurf.
7            928        smurf.
8           1795        smurf.
9              0          None
10         10360        smurf.
11         38584        smurf.
12          4221        smurf.
13         19607       normal.
14             0          None
15         14463        smurf.
16             0          None
17          9203        smurf.
18          2222        smurf.
19          3835        smurf.
20             0          None
21            62        smurf.
22          2053        smurf.
23          8340        smurf.
24          9398        smurf.
25         55494        smurf.
26          5545       normal.
27             0          None
28          6374        smurf.
29             0          None
30         12171        

#### Testing and Evaluation

In [36]:
def test_kmeans( k, test_data, centroids, cluster_labels ):
    
    y_pred = []
    cluster_labels = pd.Series(cluster_labels)
    cluster_labels = cluster_labels.apply(lambda x: 'anomaly.' if x != 'normal.' else x)

    for i in range( test_data.shape[0] ):

        # Calculating the distance of the data point from each centroid
        distances = [ np.linalg.norm( test_data.iloc[i] - centroids.iloc[j] ) for j in range(k) ]

        # Finding the index of the centroid with the minimum distance
        min_index = np.argmin(distances)

        # Assigning the label of the centroid to the data point
        y_pred.append( cluster_labels[min_index] )

    return y_pred

# This function will be used to evaluate the model using sklearn's functions
def evaluate_model( y_test, y_pred, avg ):
        precision = precision_score( y_test, y_pred, average=avg )
        recall = recall_score( y_test, y_pred, average=avg )
        f1 = f1_score( y_test, y_pred, average=avg )
        accuracy = accuracy_score( y_test, y_pred )
    
        return precision, recall, f1, accuracy

In [24]:
y_test = pd.Series(y_test)
y_test = y_test.apply(lambda x: 'anomaly.' if x != 'normal.' else x)

In [25]:
y_pred7 = test_kmeans( k=7, test_data=X_test, centroids=centroids7, cluster_labels=cluster_labels7 )

In [26]:
precision7_macro, recall7_macro, f1_7_macro, accuracy7_macro = evaluate_model( y_test, y_pred7, avg='macro' )
precision7_weighted, recall7_weighted, f1_7_weighted, accuracy7_weighted = evaluate_model( y_test, y_pred7, avg='weighted' )

In [43]:
print( 'K = 7:' )
print( '------' )

print( 'Precision (macro): ', precision7_macro )
print( 'Recall (macro): ', recall7_macro )
print( 'F1 Score (macro): ', f1_7_macro )
print( 'Accuracy: ', accuracy7_macro )
print( '-' * 50 )

print( 'Precision (weighted): ', precision7_weighted )
print( 'Recall (weighted): ', recall7_weighted )
print( 'F1 Score (weighted): ', f1_7_weighted )
print( 'Accuracy: ', accuracy7_weighted )
print( '-' * 50 )

K = 7:
------
Precision (macro):  0.8580973865410653
Recall (macro):  0.9492501916556524
F1 Score (macro):  0.8915295430078155
Accuracy:  0.9226760205639989
--------------------------------------------------
Precision (weighted):  0.9435369027394036
Recall (weighted):  0.9226760205639989
F1 Score (weighted):  0.9270071164848596
Accuracy:  0.9226760205639989
--------------------------------------------------


In [28]:
print( 'Number of detected anomalies: ', y_pred7.count('anomaly.'), 'out of', y_test.value_counts()['anomaly.'] )

Number of detected anomalies:  227260 out of 250436


In [29]:
y_pred15 = test_kmeans( k=15, test_data=X_test, centroids=centroids15, cluster_labels=cluster_labels15 )

In [30]:
precision15_macro, recall15_macro, f1_15_macro, accuracy15_macro = evaluate_model( y_test, y_pred15, avg='macro' )
precision15_weighted, recall15_weighted, f1_15_weighted, accuracy15_weighted = evaluate_model( y_test, y_pred15, avg='weighted' )

In [41]:
print( 'K = 15:' )
print( '-------' )

print( 'Precision (macro): ', precision15_macro )
print( 'Recall (macro): ', recall15_macro )
print( 'F1 Score (macro): ', f1_15_macro )
print( 'Accuracy: ', accuracy15_macro )
print( '-' * 50 )

print( 'Precision (weighted): ', precision15_weighted )
print( 'Recall (weighted): ', recall15_weighted )
print( 'F1 Score (weighted): ', f1_15_weighted )
print( 'Accuracy: ', accuracy15_weighted )
print( '-' * 50 )

K = 15:
-------
Precision (macro):  0.8791609001967897
Recall (macro):  0.9574728612385373
F1 Score (macro):  0.9101168090835648
Accuracy:  0.9374688533866617
--------------------------------------------------
Precision (weighted):  0.9513638155991788
Recall (weighted):  0.9374688533866617
F1 Score (weighted):  0.9403809144749531
Accuracy:  0.9374688533866617
--------------------------------------------------


In [32]:
print( 'Number of detected anomalies: ', y_pred15.count('anomaly.'), 'out of', y_test.value_counts()['anomaly.'] )

Number of detected anomalies:  232169 out of 250436


In [37]:
y_pred23 = test_kmeans( k=23, test_data=X_test, centroids=centroids23, cluster_labels=cluster_labels23 )

In [38]:
precision23_macro, recall23_macro, f1_23_macro, accuracy23_macro = evaluate_model( y_test, y_pred23, avg='macro' )
precision23_weighted, recall23_weighted, f1_23_weighted, accuracy23_weighted = evaluate_model( y_test, y_pred23, avg='weighted' )

In [44]:
print( 'K = 23:' )
print( '-------' )

print( 'Precision (macro): ', precision23_macro )
print( 'Recall (macro): ', recall23_macro )
print( 'F1 Score (macro): ', f1_23_macro )
print( 'Accuracy: ', accuracy23_macro )
print( '-' * 50 )

print( 'Precision (weighted): ', precision23_weighted )
print( 'Recall (weighted): ', recall23_weighted )
print( 'F1 Score (weighted): ', f1_23_weighted )
print( 'Accuracy: ', accuracy23_weighted )
print( '-' * 50 )

K = 23:
-------
Precision (macro):  0.8702070083195891
Recall (macro):  0.9552373098396587
F1 Score (macro):  0.9026856433086805
Accuracy:  0.9315015641628273
--------------------------------------------------
Precision (weighted):  0.9484834592431775
Recall (weighted):  0.9315015641628273
F1 Score (weighted):  0.9350076572685848
Accuracy:  0.9315015641628273
--------------------------------------------------


In [40]:
print( 'Number of detected anomalies: ', y_pred23.count('anomaly.'), 'out of', y_test.value_counts()['anomaly.'] )

Number of detected anomalies:  229843 out of 250436


In [ ]:
y_pred31 = test_kmeans( k=31, test_data=X_test, centroids=centroids31, cluster_labels=cluster_labels31 )

In [ ]:
precision31_macro, recall31_macro, f1_31_macro, accuracy31_macro = evaluate_model( y_test, y_pred31, avg='macro' )
precision31_weighted, recall31_weighted, f1_31_weighted, accuracy31_weighted = evaluate_model( y_test, y_pred31, avg='weighted' )

In [ ]:
print( 'K = 31:' )
print( '-------' )

print( 'Precision (macro): ', precision31_macro )
print( 'Recall (macro): ', recall31_macro )
print( 'F1 Score (macro): ', f1_31_macro )
print( 'Accuracy: ', accuracy31_macro )
print( '-' * 50 )

print( 'Precision (weighted): ', precision31_weighted )
print( 'Recall (weighted): ', recall31_weighted )
print( 'F1 Score (weighted): ', f1_31_weighted )
print( 'Accuracy: ', accuracy31_weighted )
print( '-' * 50 )

K = 31:
-------
Precision (macro):  0.40458939936529364
Recall (macro):  0.4854709059178062
F1 Score (macro):  0.4395095272929569
Accuracy:  0.7811233036147754
--------------------------------------------------
Precision (weighted):  0.6462494770010856
Recall (weighted):  0.7811233036147754
F1 Score (weighted):  0.7065920746048052
Accuracy:  0.7811233036147754
--------------------------------------------------


In [ ]:
print( 'Number of detected anomalies: ', y_pred31.count('anomaly.'), 'out of', y_test.value_counts()['anomaly.'] )

Number of detected anomalies:  303413 out of 250436


In [ ]:
y_pred45 = test_kmeans( k=45, test_data=X_test, centroids=centroids45, cluster_labels=cluster_labels45 )

In [ ]:
precision45_macro, recall45_macro, f1_45_macro, accuracy45_macro = evaluate_model( y_test, y_pred45, avg='macro' )
precision45_weighted, recall45_weighted, f1_45_weighted, accuracy45_weighted = evaluate_model( y_test, y_pred45, avg='weighted' )

In [ ]:
print( 'K = 45:' )
print( '-------' )

print( 'Precision (macro): ', precision45_macro )
print( 'Recall (macro): ', recall45_macro )
print( 'F1 Score (macro): ', f1_45_macro )
print( 'Accuracy: ', accuracy45_macro )
print( '-' * 50 )

print( 'Precision (weighted): ', precision45_weighted )
print( 'Recall (weighted): ', recall45_weighted )
print( 'F1 Score (weighted): ', f1_45_weighted )
print( 'Accuracy: ', accuracy45_weighted )
print( '-' * 50 )

In [ ]:
print( 'Number of detected anomalies: ', y_pred45.count('anomaly.'), 'out of', y_test.value_counts()['anomaly.'] )